Asteroid identification 3/3

Part of the photobomb package to measure asteroid diameters from serendipitous detections with JWST MIRI. </br>
This notebook: verify which candidates actually intersect a datafile using the WCS, calculate motion rates in x/y pixels/frame. <br />
M.Mueller@astro.rug.nl, 2022/10/06+

In [1]:
import os
import json
import numpy as np
from astroquery.jplhorizons import Horizons
from glob import glob
from jwst.assign_wcs import AssignWcsStep
#from jwst.pipeline import calwebb_detector1
from jwst import datamodels

In [3]:
pid=1051
obs=19
### make that a parameter to a function once stuff works

In [4]:
basedirData = os.path.join( os.environ['HOME'], 'MIRI', 'data', 'photobomb')
basedirResults = os.path.join( os.environ['HOME'], 'ownCloud', 'miri', 'photobomb' )
for d in [basedirData, basedirResults ]:
    if not os.path.isdir( d ):
        os.makedirs( d )

In [5]:
dataDir = os.path.join( basedirData, '%s'%pid, '%s'%obs )
resultsDir = os.path.join( basedirResults, '%s'%pid, '%s'%obs )
for d in [dataDir, resultsDir ]:
    if not os.path.isdir( d ):
        os.makedirs( d )

In [6]:
telescope='@JWST'
elongLimits = [85,135]

In [7]:
candidatesFile = os.path.join( resultsDir, 'candidates2.json' )
#if  os.path.isfile( candidatesFile ):
if not os.path.isfile( candidatesFile ):
    print( candidatesFile, "doesn't exist, run earlier notebooks first")
    assert False
else:
    with open( candidatesFile ) as f:
        parms = json.load( f )

In [8]:
### Bootstrap: file doesn't exist, yet. Set up dict manually (to be deleted once notebooks 1 and 2 exist)
#parms={}
#parms['pid']=pid
#parms['obs']=obs
#parms['DATE-BEG']= '2022-05-31T15:16:24.193'
#parms['DATE-END']= '2022-05-31T15:16:46.393'
#parms['TARG_RA'] = 182.34533
#parms['TARG_DEC']=3.989880555555556 

#candidates = {}
#candidates['2018 HP10']='2022-10-01T00:00:00.000'
#parms['candidates']=candidates

#assert not os.path.isfile( candidatesFile ) # don't overwrite
#with open( candidatesFile, 'w' ) as f:
#    f.write(json.dumps(parms, indent=4))

In [9]:
parms

{'pid': 1051,
 'obs': 19,
 'DATE-BEG': '2022-05-31T15:16:24.193',
 'DATE-END': '2022-05-31T15:16:46.393',
 'TARG_RA': 182.34533,
 'TARG_DEC': 3.989880555555556,
 'candidates': {'2018 HP10': '2022-10-01T00:00:00.000'}}

In [10]:
#from datetime import datetime

#timeFormat='%Y-%m-%dT%H:%M:%S.%f'
#t0=datetime.strptime( parms['DATE-BEG'], timeFormat )
#t1=datetime.strptime( parms['DATE-END'], timeFormat )

#dT = (t1-t0)/(nTimes-0)
#times=np.arange( t0, t1, dT )
#times=np.append(times,t1) # add end time

In [11]:
if len(parms['candidates']) == 0:
    #return # only makes sense within a function
    pass

In [22]:
rateFiles=glob(os.path.join(dataDir, '*_rate.fits'))

In [63]:
detections=[]
for rateFile in rateFiles:
    print( rateFile )
    try:
        dm = datamodels.ImageModel( rateFile )
    except:
        print( "Couldn't open file", rateFile)
        print( "Moving on")
        continue
    dm.meta.exposure.type = 'MIR_IMAGE' # needed for some commissioning observations (pipeline doesn't run otherwise)
    rateWithWCS = AssignWcsStep().call( dm )

    coordToPix = rateWithWCS.meta.wcs.get_transform( 'world', 'detector' )

    #nGroups = round(rateWithWCS.meta.exposure.elapsed_exposure_time / rateWithWCS.meta.exposure.frame_time)
    nGroups=rateWithWCS.meta.exposure.ngroups

    epochs={}
    epochs['start']=parms['DATE-BEG']
    epochs['stop'] =parms['DATE-END']
    epochs['step'] = '%i'%(nGroups-1) 

    for c in parms['candidates']:
        # check candidate timestamp against output file (if already present)
        hor=Horizons( id=c, location=telescope, epochs=epochs )
        try:
            ephemeris = hor.ephemerides(solar_elongation=elongLimits) 
        except ValueError:
            print( "Couldn't find ",c, ', packed number?  This issue really should be fixed now!')
            ### Those don't appear to be supported by astroquery.jplHorizons as of 2019/05/13 (but compressed desigs are)
            ### That's a problem!
            continue
    # Everything below should be part of the for loop above (run it once per candidate).  Current situation is for development, only

    #print( ephemeris )

    ra=ephemeris['RA']
    dec=ephemeris['DEC']

    x,y = coordToPix( ra, dec )

    xMax,yMax=rateWithWCS.data.shape  # or should it be y,x???

    dx= (x[-1]-x[0]) / nGroups
    dy= (y[-1]-y[0]) / nGroups

    inside = [True for i in range(nGroups)]
    for i in range(nGroups):
        if (x[i] < 0) or (y[i]<0) or (x[i]>xMax) or (y[i]>yMax):
            inside[i]=False
    print( rateFile, True in inside)
    if True in inside:
        detections.append(rateFiles)

2022-10-24 16:01:33,690 - stpipe.AssignWcsStep - INFO - AssignWcsStep instance created.
2022-10-24 16:01:33,703 - stpipe.AssignWcsStep - INFO - AssignWcsStep instance created.


/Users/michaelm/MIRI/data/photobomb/1051/19/jw01051019001_02105_00001_mirimage_rate.fits


2022-10-24 16:01:34,041 - stpipe.AssignWcsStep - INFO - Step AssignWcsStep running with args (<ImageModel(1024, 1032) from jw01051019001_02105_00001_mirimage_rate.fits>,).
2022-10-24 16:01:34,046 - stpipe.AssignWcsStep - INFO - Step AssignWcsStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'sip_approx': True, 'sip_max_pix_error': 0.1, 'sip_degree': None, 'sip_max_inv_pix_error': 0.1, 'sip_inv_degree': None, 'sip_npoints': 12, 'slit_y_low': -0.55, 'slit_y_high': 0.55}
2022-10-24 16:01:34,379 - stpipe.AssignWcsStep - INFO - Created a MIRI mir_image pipeline with references {'distortion': '/Users/michaelm/.miricle/crds/references/jwst/miri/jwst_miri_distortion_0047.asdf', 'filteroffset': '/Users/michaelm/.miricle/crds/references/jwst/miri/jwst_miri_filteroffset_0006.as

/Users/michaelm/MIRI/data/photobomb/1051/19/jw01051019001_02105_00001_mirimage_rate.fits False
/Users/michaelm/MIRI/data/photobomb/1051/19/jw01051019001_0210a_00001_mirimage_rate.fits


2022-10-24 16:01:35,146 - stpipe.AssignWcsStep - INFO - Step AssignWcsStep running with args (<ImageModel(1024, 1032) from jw01051019001_0210a_00001_mirimage_rate.fits>,).
2022-10-24 16:01:35,150 - stpipe.AssignWcsStep - INFO - Step AssignWcsStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'sip_approx': True, 'sip_max_pix_error': 0.1, 'sip_degree': None, 'sip_max_inv_pix_error': 0.1, 'sip_inv_degree': None, 'sip_npoints': 12, 'slit_y_low': -0.55, 'slit_y_high': 0.55}
2022-10-24 16:01:35,498 - stpipe.AssignWcsStep - INFO - Created a MIRI mir_image pipeline with references {'distortion': '/Users/michaelm/.miricle/crds/references/jwst/miri/jwst_miri_distortion_0047.asdf', 'filteroffset': '/Users/michaelm/.miricle/crds/references/jwst/miri/jwst_miri_filteroffset_0006.as

...

/Users/michaelm/MIRI/data/photobomb/1051/19/jw01051019001_0210c_00001_mirimage_rate.fits False
/Users/michaelm/MIRI/data/photobomb/1051/19/jw01051019001_0210c_00002_mirimage_rate.fits


2022-10-24 16:01:52,657 - stpipe.AssignWcsStep - INFO - Step AssignWcsStep running with args (<ImageModel(1024, 1032) from jw01051019001_0210c_00002_mirimage_rate.fits>,).
2022-10-24 16:01:52,660 - stpipe.AssignWcsStep - INFO - Step AssignWcsStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'sip_approx': True, 'sip_max_pix_error': 0.1, 'sip_degree': None, 'sip_max_inv_pix_error': 0.1, 'sip_inv_degree': None, 'sip_npoints': 12, 'slit_y_low': -0.55, 'slit_y_high': 0.55}
2022-10-24 16:01:52,989 - stpipe.AssignWcsStep - INFO - Created a MIRI mir_image pipeline with references {'distortion': '/Users/michaelm/.miricle/crds/references/jwst/miri/jwst_miri_distortion_0047.asdf', 'filteroffset': '/Users/michaelm/.miricle/crds/references/jwst/miri/jwst_miri_filteroffset_0006.as

/Users/michaelm/MIRI/data/photobomb/1051/19/jw01051019001_0210c_00002_mirimage_rate.fits False
/Users/michaelm/MIRI/data/photobomb/1051/19/jw01051019001_02103_00002_mirimage_rate.fits


2022-10-24 16:01:53,762 - stpipe.AssignWcsStep - INFO - Step AssignWcsStep running with args (<ImageModel(1024, 1032) from jw01051019001_02103_00002_mirimage_rate.fits>,).
2022-10-24 16:01:53,766 - stpipe.AssignWcsStep - INFO - Step AssignWcsStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'sip_approx': True, 'sip_max_pix_error': 0.1, 'sip_degree': None, 'sip_max_inv_pix_error': 0.1, 'sip_inv_degree': None, 'sip_npoints': 12, 'slit_y_low': -0.55, 'slit_y_high': 0.55}
2022-10-24 16:01:54,066 - stpipe.AssignWcsStep - INFO - Created a MIRI mir_image pipeline with references {'distortion': '/Users/michaelm/.miricle/crds/references/jwst/miri/jwst_miri_distortion_0047.asdf', 'filteroffset': '/Users/michaelm/.miricle/crds/references/jwst/miri/jwst_miri_filteroffset_0006.as

/Users/michaelm/MIRI/data/photobomb/1051/19/jw01051019001_02103_00002_mirimage_rate.fits False


In [64]:
detections

[]